<a href="https://colab.research.google.com/github/6RobinLiu/PytorchDL/blob/main/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Create Full Connected Network 全连接层

In [2]:
class NN(nn.Module):
  def __init__(self,input_size,num_classes):
    super().__init__()
    self.fc1=nn.Linear(input_size,50)
    self.fc2=nn.Linear(50,num_classes)

  def forward(self,x):
    x=F.relu(self.fc1(x))
    x=self.fc2(x)
    return x

model=NN(784,10)
x=torch.randn(64,784)
print(model(x).shape)

torch.Size([64, 10])


# Set Device

In [3]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyperparameters

In [4]:
input_size=784
num_classess=10
learning_rate=0.001
batch_size=64
num_epochs=1

# Load Data

In [5]:
train_dataset=datasets.MNIST(root='dataset/',train=True,transform=transforms.ToTensor(),download=True)
train_loader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataset=datasets.MNIST(root='dataset/',train=False,transform=transforms.ToTensor(),download=True)
test_loader=DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)


100%|██████████| 9.91M/9.91M [00:00<00:00, 15.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 532kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.85MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.68MB/s]


# Loss and optimizer


In [6]:
criterion=nn.CrossEntropyLoss()
optimizier=optim.Adam(model.parameters(),lr=learning_rate)

# Train Network

In [7]:
for epoch in range(num_epochs):
  for batch_idx,(data,targets) in enumerate(train_loader):
    # get data to cuda
    data=data.to(device=device)
    targets=targets.to(device=device)

    # get to correct shape
    data=data.reshape(data.shape[0],-1)

    #forward
    scores=model(data)
    loss=criterion(scores,targets)

    #backward
    optimizier.zero_grad()
    loss.backward()

    #gradient descent or adam step
    optimizier.step()

# Check accuracy

In [9]:
def check_accuracy(loader,model):
  if loader.dataset.train:
    print('Checking accuracy on training data')
  else:
    print('Checking accuracy on test data')
  num_correct=0
  num_samples=0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x=x.to(device=device)
      y=y.to(device=device)
      x=x.reshape(x.shape[0],-1)

      scores=model(x)
      _,predictions=scores.max(1)
      num_correct+=(predictions==y).sum()
      num_samples+=predictions.size(0)
    print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

  model.train()

check_accuracy(train_loader,model)
check_accuracy(test_loader,model)

Checking accuracy on training data
Got 55832/60000 with accuracy 93.05
Checking accuracy on test data
Got 9323/10000 with accuracy 93.23
